<h1 align = "center">机器学习第六次实验报告</h1>

<h5 align = "center">姓名：邢清画&nbsp&nbsp&nbsp&nbsp         学号：2211999  &nbsp &nbsp&nbsp专业：物联网工程</h5>

### 实验题目

- 决策树分类

### 实验要求

#### 1. 基本要求：

1. 基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
2. 基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；


#### 2. 中级要求：

1. 对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
2. 对测试集Watermelon-test2进行预测，输出分类精度；


#### 3. 高级要求：

使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

#### 数据集

Watermelon-test1/2、Watermelon-train1/2

### **代码实现**

### **1. 基本要求**

### **ID3算法基本原理**
ID3（Iterative Dichotomiser 3）算法是决策树算法的一种，它通过选择信息增益最大的特征来划分数据集。ID3 是一种贪心算法，即在每一步选择当前最优的划分特征，而不考虑之后的步骤。ID3 通常用于分类问题，适用于离散属性的数据。
#### **主要步骤**

1. 选择最佳特征：对于每一个特征，计算其信息增益。信息增益越大，说明该特征对分类的贡献越大，选择该特征作为划分标准。
2. 构建决策树：递归地选择最佳特征进行划分，直到数据集中的样本属于同一类别或者没有更多的特征可以划分为止。
3. 停止条件：
* 所有样本属于同一类别。
* 没有剩余特征可以进行划分。
* 数据集为空。

### **1.1 关键内容**

#### **数据初始化**
加载一个包含特征和标签的文本数据集，并将离散的特征值和标签映射为数值索引，以便于后续处理。

通过定义特征字典 feature_dict 和标签列表 label_list，实现了从文本文件读取数据并逐行解析，将特征值转为对应的索引形式，同时保留样本 ID 和标签索引，最终返回一个 NumPy 数组格式的数据集，便于后续建模或分析。

In [164]:
import numpy as np
import codecs

# 特征和标签定义
feature_dict = {"色泽": ["青绿", "乌黑", "浅白"],
                "根蒂": ["蜷缩", "稍蜷", "硬挺"],
                "敲声": ["浊响", "沉闷", "清脆"],
                "纹理": ["清晰", "稍糊", "模糊"]
                }
label_list = ["否", "是"]
feature_list = ["色泽", "根蒂", "敲声", "纹理"]

def load_txt(path):
    """加载数据集并进行预处理"""
    ans = []
    with codecs.open(path, "r", "GBK") as f:
        line = f.readline()
        line = f.readline()
        while line:
            d = line.rstrip("\r\n").split(',')
            re = []
            re.append(int(d[0]))
            re.append(feature_dict.get("色泽").index(d[1]))
            re.append(feature_dict.get("根蒂").index(d[2]))
            re.append(feature_dict.get("敲声").index(d[3]))
            re.append(feature_dict.get("纹理").index(d[4]))
            re.append(label_list.index(d[-1]))
            ans.append(np.array(re))
            line = f.readline()
    return np.array(ans)

将离散的特征值和标签映射0，1，2这样的数值索引。

#### **决策树的构建**

定义了一个 Node 类，用于构建决策树的节点，并提供了一个辅助函数 is_same_on_attr 来判断给定数据集在某一属性上的值是否相同。
#### **1. Node 类**

定义了决策树中的每个节点，包含了属性、标签、子节点等信息。每个节点有以下几个重要成员变量：

* attr：表示该节点的划分属性（即用于划分数据集的特征）。例如，在决策树的构建中，attr 可能是“色泽”或“根蒂”等特征。

* label：表示该节点的类别标签。如果该节点是一个叶节点，则 label 存储的是该类的标签（例如 "是" 或 "否"）。如果该节点不是叶节点，则 label 可以为 None，因为它用于指示尚未决定的类别。

* attr_v：表示该节点对应的属性取值。具体而言，attr_v 表示当前节点划分数据集时，该特征的具体值（例如，对于“色泽”特征，attr_v 可能是“青绿”、“乌黑”或“浅白”）。

* children：一个列表，用于存储当前节点的子节点。每个子节点代表了当前节点基于某一特征值进行划分后的子集。

* depth：表示当前节点在决策树中的深度。根节点的深度为 0，根节点的子节点深度为 1，依此类推。

In [165]:
class Node:
    def __init__(self, attr, label, v):
        self.attr = attr      # 节点的划分属性
        self.label = label    # 节点的类别标签
        self.attr_v = v       # 属性取值
        self.children = []    # 子节点列表
        self.depth = 0        # 节点深度
        self.split_value = None  # 添加split_value属性，用于C4.5

#### **2. is_same_on_attr 函数**

该函数用于检查给定数据集 X 中，在特定属性 attrs 上，所有样本的属性值是否相同。

X_a = X[:, attrs]：从数据集 X 中提取出所有样本在指定特征（attrs）上的值。

target = X_a[0]：选取数据集第一行（第一个样本）该特征的值作为目标值 target。

然后使用 for 循环遍历数据集中的每一行，逐个比较当前样本在该特征上的值是否与 target 相同。如果发现任何一个样本的该特征值与 target 不同，返回 False，表示该属性值不一致；如果所有样本在该属性上的值都相同，则返回 True。

In [166]:
def is_same_on_attr(X, attrs):
    """验证属性上的值是否均相同"""
    X_a = X[:, attrs]
    target = X_a[0]
    for r in range(X_a.shape[0]):
        row = X_a[r]
        if (row != target).any():
            return False
    return True

#### **3. 信息熵的计算**

信息熵（Entropy）是信息理论中的一个重要概念，用来衡量一个随机变量的不确定性或信息量。在机器学习中，信息熵常用于决策树的构建（如 ID3 算法）来选择最佳的划分特征。

**信息熵的定义**： 假设有一个随机变量 $D$，它可以取 $k$ 个不同的值（例如分类问题中的标签类别）。信息熵 $H(D)$ 表示随机变量 $D$ 所包含的信息量，计算公式如下：

$$
H(D) = -\sum_{k=1}^{K} p_k \log_2(p_k)
$$

其中：

*   $D$ 是样本集（或数据集），表示一组数据的分类结果或属性值。
*   $\log_2(p_k)$ 是类别 $k$ 的信息量，表示 $p_k$ 发生的概率所包含的信息量。
*   $p_k$ 是随机变量 $D$ 取值为类别 $k$ 的概率，计算公式为：

$$
p_k = \frac{\text{类别} \, k \, \text{的样本数}}{\text{总样本数}}
$$

*   $\log_2(p_k)$ 是以 2 为底的对数，用来衡量该类别的信息量。
*   信息熵是所有类别的信息量的加权平均，权重就是每个类别出现的概率 $p_k$。

#### 信息熵的解释

*   当所有样本均匀分布在各个类别上时，熵值最大，表示系统的不确定性最大。
*   如果某一个类别的样本占绝大多数，而其他类别的样本非常少，熵值接近于 0，表示系统的不确定性较低，模型很容易进行分类。

#### 信息熵计算步骤

1.  **计算每个类别的概率**： 对于每个类别 $k$，计算该类别样本数占总样本数的比例，得到 $p_k$。
2.  **计算每个类别的信息量**：对于每个类别，使用公式 $-p_k \log_2(p_k)$ 计算其信息量。
3.  **求和**：将所有类别的信息量加权求和，得到总的熵值。
4.  **取负值**：最后，取加权和的负值，得到最终的信息熵。

In [167]:
def ent(D):
    """计算信息熵"""
    s = 0
    for k in set(D):
        p_k = np.sum(np.where(D == k, 1, 0)) / np.shape(D)[0]
        if p_k == 0:
            continue
        s += p_k * np.log2(p_k)
    return -s

#### **4. 信息增益的计算**

信息增益（Information Gain）是决策树算法中用来选择最佳划分特征的重要指标。它衡量的是在给定特征的条件下，数据集的不确定性（信息熵）减少的程度。在构建决策树时，算法会选择使信息增益最大的特征作为划分依据。

**信息增益的定义**：

给定一个数据集 $D$ 和某个特征 $A$，信息增益衡量的是使用特征 $A$ 对数据集 $D$ 进行划分后，系统的不确定性减少了多少。

信息增益的计算步骤包括：

1.  **计算特征划分前的熵（$H(D)$）**：衡量数据集的原始不确定性。
2.  **计算特征划分后的条件熵**：特征 $A$ 将数据集划分为若干子集，对于每个子集，计算熵 $H(D_v)$，其中 $D_v$ 是特征 $A$ 取值为 $v$ 的子集。
3.  **加权平均计算条件熵**：每个子集的熵加权后求和，权重为该子集在总数据集中的比例。
4.  **计算信息增益**：信息增益是原始熵与条件熵之间的差值，公式如下：

#### 数学公式

给定数据集 $D$ 和特征 $A$，其信息增益 $Gain(D, A)$ 可以通过以下公式计算：

$$
Gain(D, A) = H(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} H(D_v)
$$

其中：

- $H(D)$ 是数据集 $D$ 的熵，表示未使用任何特征前数据集的纯度或不确定性。
  
  $$
  H(D) = - \sum_{k=1}^{K} p_k \log_2(p_k)
  $$

  其中 $p_k$ 是标签 $k$ 的概率。
  
- $H(D_v)$ 是数据集 $D$ 在特征 $A$ 的某一取值 $v$ 上的熵，表示在特征 $A$ 取值为 $v$ 时，数据集的纯度。
  
- $|D_v|$ 是特征 $A$ 取值为 $v$ 的样本数量，$|D|$ 是数据集 $D$ 的总样本数量。
  
- $\frac{|D_v|}{|D|}$ 是特征 $A$ 取值为 $v$ 的子集的权重。

#### 计算步骤

1.  **计算数据集的总熵**：首先，计算数据集 $D$ 的熵，$H(D)$，通过其类别标签的概率分布来衡量数据集的纯度。
    
2.  **按特征 $A$ 的取值划分数据集**：对于特征 $A$，根据其所有可能的取值，将数据集划分为多个子集 $D_v$，其中每个子集对应特征 $A$ 的一个取值。
    
3.  **计算每个子集的熵**：计算每个子集 $D_v$ 的熵 $H(D_v)$，表示在该子集中的类别分布的不确定性。
    
4.  **加权求和**：通过加权求和，得到条件熵，权重为每个子集在总数据集中的比例。
    
5.  **计算信息增益**：信息增益 $Gain(D, A)$ 等于总熵减去条件熵。

In [168]:
def gain(X, Y, attr):
    """计算信息增益"""
    x_attr_col = X[:, attr]
    ent_Dv = []
    weight_Dv = []
    for x_v in set(x_attr_col):
        index_x_equal_v = np.where(x_attr_col == x_v)
        y_x_equal_v = Y[index_x_equal_v]
        ent_Dv.append(ent(y_x_equal_v))
        weight_Dv.append(np.shape(y_x_equal_v)[0] / np.shape(Y)[0])
    return ent(Y) - np.sum(np.array(ent_Dv) * np.array(weight_Dv))

在代码中，gain 函数通过以下步骤计算信息增益：

1.  **提取特征列**：通过 `X[:, attr]` 获取特征列 `x_attr_col`，这是数据集 $D$ 中所有样本的特征 $A$ 的值。
2.  **遍历特征的所有取值**：使用 `set(x_attr_col)` 获取特征 $A$ 的所有不同取值，并对每个取值 $x_v$ 计算条件熵。
3.  **计算条件熵**：
    *   使用 `np.where(x_attr_col == x_v)` 找出特征列中等于当前取值 $x_v$ 的样本的索引。
    *   根据这些索引从标签 $Y$ 中提取对应的标签子集 y_x_equal_v。
    *   调用 `ent(y_x_equal_v)` 计算该子集的熵。
    *   计算该子集的权重，即该子集样本数占总样本数的比例。
4.  **计算信息增益**：最后，使用总熵减去加权平均的条件熵，得到信息增益。

#### **5. 构建决策树代码**

通过递归的方式构建决策树。根据给定的特征和标签数据，ID3算法在每一步选择信息增益最大的特征作为当前节点的划分属性，并递归地对子节点进行相同的处理，直到满足停止条件为止。

**1.递归停止条件：**

* 类别唯一性：如果当前节点的数据集中的标签 $Y$ 只有一个类别，说明当前节点已经是一个叶节点，直接将该类别作为该节点的标签。
 特征用尽或特征值相同：如果所有特征已经用尽，或者当前特征的所有样本值都相同（即所有数据的某个特征值相同），也视为停止递归。此时，节点的标签被设为 $Y$ 中出现最多的类别。

**2.计算信息增益：** 

在每一层，函数遍历所有特征，并计算每个特征的信息增益。信息增益越大，代表该特征对分类的贡献越大，选择信息增益最大的特征作为当前节点的划分属性。

**3.创建节点并递归构建子树：**

* 选择最优特征：通过比较所有特征的信息增益，选择信息增益最大（即划分效果最佳）的特征作为当前节点的划分依据。
* 划分数据集：根据当前特征的不同取值将数据集分成若干子集。对于每个子集，创建一个子节点并递归调用该函数，继续进行划分。

In [169]:
def dicision_tree_init(X, Y, attrs, root, purity_cal):
    """构建决策树"""
    if len(set(Y)) == 1:
        root.attr = np.pi
        root.label = Y[0]
        return None

    if len(attrs) == 0 or is_same_on_attr(X, attrs):
        root.attr = np.pi
        root.label = np.argmax(np.bincount(Y))
        return None

    purity_attrs = []
    for i, a in enumerate(attrs):
        p = purity_cal(X, Y, a)
        purity_attrs.append(p)
    
    chosen_index = purity_attrs.index(max(purity_attrs))
    chosen_attr = attrs[chosen_index]

    root.attr = chosen_attr
    root.label = np.pi

    remaining_attrs = attrs.copy()
    del remaining_attrs[chosen_index]

    x_attr_col = X[:, chosen_attr]
    for x_v in set(X[:, chosen_attr]):
        n = Node(-1, -1, x_v)
        root.children.append(n)

        index_x_equal_v = np.where(x_attr_col == x_v)
        X_x_equal_v = X[index_x_equal_v]
        Y_x_equal_v = Y[index_x_equal_v]
        dicision_tree_init(X_x_equal_v, Y_x_equal_v, remaining_attrs, n, purity_cal)

#### **可视化树的层级和节点信息**

它通过递归遍历决策树的每个节点，并按照特定格式输出树的结构，标明每个节点的类别标签、特征及其取值。可以清晰地看到决策树的层次和每个节点的特征划分，方便理解决策过程。

In [170]:
def print_tree(node, prefix="", is_last=True):
    """打印决策树结构"""
    if node.label != np.pi:  # 叶节点
        print(prefix + ("└── " if is_last else "├── ") + f"[叶节点]Label: {label_list[node.label]}")
        return

    # 内部节点
    print(prefix + ("└── " if is_last else "├── ") + f"[内部节点]Feature: {feature_list[node.attr]}")
    
    for i, child in enumerate(node.children):
        is_last_child = (i == len(node.children) - 1)
        new_prefix = prefix + ("    " if is_last else "│   ")
        
        # 打印属性值
        print(new_prefix + ("└── " if is_last_child else "├── ") + 
              f"Value: {feature_dict[feature_list[node.attr]][child.attr_v]}")
        
        # 递归打印子树
        print_tree(child, new_prefix + "    ", is_last_child)

#### **使用决策树进行预测**

用构建好的决策树进行预测。根据给定的特征向量 x 和决策树的根节点 tree_root，函数递归地遍历树的节点，依据特征值选择路径，直到达到叶节点并返回预测结果。
#### 关键步骤：

* 叶节点判断：通过判断 tree_root.label != np.pi，识别是否是叶节点。如果是叶节点，则直接返回该节点的标签 label。
* 递归子节点：若当前节点是内部节点，获取当前节点的划分特征 chose_attr，并根据输入样本 x 的该特征的取值选择对应的子节点。
* 递归调用：对于符合条件的子节点，递归调用 dicision_tree_predict 函数，直到达到叶节点。

In [171]:
def dicision_tree_predict(x, tree_root):
    """使用决策树进行预测"""
    if tree_root.label != np.pi:
        return tree_root.label

    if tree_root.label == np.pi and tree_root.attr == np.pi:
        print("err!")
        return None

    chose_attr = tree_root.attr
    for child in tree_root.children:
        if child.attr_v == x[chose_attr]:
            return dicision_tree_predict(x, child)
    return None

In [172]:
if __name__ == '__main__':
    # 1. 加载数据
    print("Loading data...")
    ans = load_txt("Watermelon-train1.csv")
    X_train = ans[:, 1: -1]
    Y_train = ans[:, -1].astype(np.int64)
    test_data = load_txt("Watermelon-test1.csv")
    X_test = test_data[:, 1:-1]
    Y_test = test_data[:, -1].astype(np.int64)

    # 打印数据集形状
    print("\nDataset Shapes:")
    print(f"Training set - Features: {X_train.shape}, Labels: {Y_train.shape}")
    print(f"Test set     - Features: {X_test.shape}, Labels: {Y_test.shape}")

    # 2. 构建决策树
    print("\nBuilding decision tree...")
    r = Node(-1, -1, -1)
    attrs = [0, 1, 2, 3]
    dicision_tree_init(X_train, Y_train, attrs, r, gain)

    # 3. 打印决策树结构
    print("\nDecision Tree Structure:")
    print_tree(r)

    # 4. 预测及评估
    print("\nPredicting on test set...")
    y_predict = []
    for i in range(X_test.shape[0]):
        x = X_test[i]
        y_p = dicision_tree_predict(x, r)
        y_predict.append(y_p)

    # 5. 输出预测结果
    print("\nPrediction Results:")
    print("ID  Actual  Predicted  Correct")
    print("-" * 35)
    correct_count = 0
    for i, (y_true, y_pred) in enumerate(zip(Y_test, y_predict)):
        is_correct = "✓" if y_true == y_pred else "✗"
        if y_true == y_pred:
            correct_count += 1
        print(f"{i+1:2d}    {label_list[y_true]:4s}    {label_list[y_pred]:4s}      {is_correct}")

    # 6. 计算并输出准确率
    accuracy = correct_count / len(Y_test)
    print(f"\nAccuracy: {accuracy:.2f}")

Loading data...

Dataset Shapes:
Training set - Features: (16, 4), Labels: (16,)
Test set     - Features: (10, 4), Labels: (10,)

Building decision tree...

Decision Tree Structure:
└── [内部节点]Feature: 纹理
    ├── Value: 清晰
        ├── [内部节点]Feature: 根蒂
        │   ├── Value: 蜷缩
        │       ├── [叶节点]Label: 是
        │   ├── Value: 稍蜷
        │       ├── [叶节点]Label: 是
        │   └── Value: 硬挺
        │       └── [叶节点]Label: 否
    ├── Value: 稍糊
        ├── [内部节点]Feature: 色泽
        │   ├── Value: 青绿
        │       ├── [叶节点]Label: 否
        │   ├── Value: 乌黑
        │       ├── [内部节点]Feature: 敲声
        │       │   ├── Value: 浊响
        │       │       ├── [叶节点]Label: 是
        │       │   └── Value: 沉闷
        │       │       └── [叶节点]Label: 否
        │   └── Value: 浅白
        │       └── [叶节点]Label: 否
    └── Value: 模糊
        └── [叶节点]Label: 否

Predicting on test set...

Prediction Results:
ID  Actual  Predicted  Correct
-----------------------------------
 1    是       是         ✓

### **1.2 结果分析**
决策树以**纹理**特征作为根节点，表明在当前数据集中，纹理的信息增益最高，最适合作为分裂的第一步。

决策树的准确率为 70%，能够在一定程度上分类测试数据，但仍有改进空间。

可能原因：

 1. 训练数据规模较小（16 个样本），决策树可能在某些特征值组合上无法完全学习数据分布，导致模型的泛化能力有限。
 2. 决策树过于依赖当前数据集的分裂逻辑，在测试集中未能完全适应某些特征组合。

 通过扩充数据集、优化模型结构或引入更强的分类算法，可以进一步提升模型的分类性能。

### **2. 中级要求**
1. 对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
2. 对测试集Watermelon-test2进行预测，输出分类精度；


### **2.1 C4.5决策树** 
C4.5算法的基本步骤如下：

1. **选择最佳分裂属性**：通过计算每个属性的信息增益率，选择信息增益率最大的属性作为当前节点的分裂属性。
2. **处理连续属性**：对于连续属性，C4.5算法通过寻找最佳的分割点，将数据集分为两个子集。
3. **递归构建子树**：对于每个子集，递归地构建子树，直到满足停止条件（如所有样本属于同一类别或没有更多的分裂点）。

##### 信息增益与信息增益率

C4.5算法使用**信息增益**来度量选择一个属性时，数据集的不确定性（熵）减少的程度。信息增益计算公式如下：

**信息增益**（Gain）计算公式：
$$
\text{Gain}(S, A) = \text{Entropy}(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \text{Entropy}(S_v)
$$
其中：
- $S$ 是数据集。
- $A$ 是某个属性。
- $\text{Values}(A)$ 是属性 $A$ 的所有可能取值。
- $S_v$ 是选择属性 $A$ 取值为 $v$ 时的子集。

**熵**（Entropy）是衡量数据集不确定性的一种方式，计算公式为：
$$
\text{Entropy}(S) = - \sum_{i=1}^{k} p_i \log_2 p_i
$$
其中：
- $p_i$ 是第 $i$ 类的概率，$k$ 是类别的总数。

**信息增益率**（Gain Ratio）则是信息增益与**分裂信息**（Split Information）的比值。信息增益率的引入是为了避免传统的ID3算法偏向于选择具有更多取值的属性。信息增益率计算公式如下：

$$
\text{Gain Ratio}(S, A) = \frac{\text{Gain}(S, A)}{\text{Split Information}(S, A)}
$$
其中，**分裂信息**的计算公式为：
$$
\text{Split Information}(S, A) = - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \log_2 \left( \frac{|S_v|}{|S|} \right)
$$
这意味着**分裂信息**度量了选择属性 $A$ 进行划分时，不同类别样本分布的均匀度。

##### 连续属性的处理

对于连续型属性，C4.5算法会尝试通过所有可能的分割点（即属性值的中点）来划分数据集。具体过程如下：
1. **排序**：将连续属性的所有值按升序排序。
2. **选择分割点**：计算相邻属性值之间的中点作为潜在的分割点。
3. **计算信息增益**：对每个分割点，分别计算分割后的信息增益。
4. **选择最佳分割点**：选择信息增益最大的分割点作为当前属性的划分点。

这种方式使得C4.5能够有效地处理连续属性，选择出最优的分割点。

#### **1. 关键内容** 

#### 1.1 计算信息增益率
这段代码的功能是计算某个属性的**信息增益比**，用于选择最佳的分割点。它处理连续型属性和离散型属性两种情况。

1. **连续属性**：
   * 对属性进行排序，计算相邻值的中点作为潜在分割点。
   * 对每个分割点，分别计算**信息增益**和**分裂信息**。
   * 信息增益比通过信息增益除以分裂信息得到，选择具有最高增益比的分割点作为最佳分割。

2. **离散属性**：
   * 对每个属性值计算信息增益和分裂信息。
   * 计算增益比并返回。

最终，代码返回最佳增益比和对应的分割点（连续属性）或 `None`（离散属性）。

In [173]:
def gain_ratio(X, Y, attr):
    """计算信息增益比"""
    x_attr_col = X[:, attr]
    if np.issubdtype(x_attr_col.dtype, np.number):  # 连续属性
        sorted_indices = np.argsort(x_attr_col)
        x_sorted = x_attr_col[sorted_indices]
        y_sorted = Y[sorted_indices]
        split_points = (x_sorted[:-1] + x_sorted[1:]) / 2
        best_gain_ratio = -float('inf')
        best_split = None
        
        for split in split_points:
            left_mask = x_attr_col <= split
            right_mask = x_attr_col > split
            if len(Y[left_mask]) == 0 or len(Y[right_mask]) == 0:
                continue
                
            # 计算信息增益
            entropy_left = ent(Y[left_mask])
            entropy_right = ent(Y[right_mask])
            weighted_entropy = (
                len(Y[left_mask]) / len(Y) * entropy_left +
                len(Y[right_mask]) / len(Y) * entropy_right
            )
            gain = ent(Y) - weighted_entropy

            # 计算分裂信息
            split_info = (
                -len(Y[left_mask]) / len(Y) * np.log2(len(Y[left_mask]) / len(Y)) -
                len(Y[right_mask]) / len(Y) * np.log2(len(Y[right_mask]) / len(Y))
            ) if len(Y[left_mask]) > 0 and len(Y[right_mask]) > 0 else 0

            # 计算增益比
            if split_info > 0:
                gain_ratio = gain / split_info
                if gain_ratio > best_gain_ratio:
                    best_gain_ratio = gain_ratio
                    best_split = split

        return best_gain_ratio, best_split
    else:  # 离散属性
        unique_values = set(x_attr_col)
        entropy_after_split = 0
        split_info = 0
        for value in unique_values:
            subset_indices = np.where(x_attr_col == value)
            subset_y = Y[subset_indices]
            prob = len(subset_y) / len(Y)
            entropy_after_split += prob * ent(subset_y)
            split_info -= prob * np.log2(prob)
        gain = ent(Y) - entropy_after_split
        gain_ratio = gain / split_info if split_info > 0 else 0
        return gain_ratio, None

#### 1.2 构建C4.5决策树

##### **基本情况处理**
- 如果样本集 `Y` 为空，或者所有样本都属于同一类，或者没有剩余属性可以选择分裂，函数会直接为当前节点设置为叶子节点，且赋予一个默认分类。

##### **判断停止条件**
- 当所有样本属于同一类时，树的当前节点成为叶节点。
- 如果没有剩余属性可以用于分裂，或已达到最大深度，或者当前样本数不足以进一步分裂（小于 `min_samples_split`），则当前节点也会成为叶子节点，并选择最多的类别作为标签。

##### **属性选择**
- 使用信息增益比（通过 `purity_cal` 函数计算）来选择最佳的分裂属性。每个属性会计算其信息增益比，选择增益比最大的属性进行分裂。
- 对于连续属性，计算该属性的最优分割点，并将数据集分割为两部分，分别递归地构建左子树和右子树。
- 对于离散属性，直接将该属性的每个取值作为一个子树节点，递归构建每个子节点。

##### **递归构建子树**
- 对于连续属性，通过分割点将数据集分为左右两部分，分别递归构建左子树和右子树。
- 对于离散属性，遍历所有可能的属性值，对每个属性值递归地构建子树。

##### **节点分配**
- 对每个子节点（无论是离散属性的每个取值，还是连续属性的左右分割），调用 `c45_tree_init` 函数进行递归调用，继续进行属性选择和节点划分。


In [174]:
def c45_tree_init(X, Y, attrs, root, purity_cal, depth=0, max_depth=None, min_samples_split=2):
    """构建C4.5决策树"""
    # 检查Y是否为空
    if len(Y) == 0:
        root.attr = np.pi
        root.label = 1  # 默认分类
        return

    # 如果所有样本属于同一类，设置为叶节点
    if len(set(Y)) == 1:
        root.attr = np.pi
        root.label = Y[0]
        return

    # 如果没有剩余属性，或者达到最大深度，或者样本不足
    if len(attrs) == 0 or (max_depth is not None and depth >= max_depth) or len(Y) < min_samples_split:
        root.attr = np.pi
        root.label = np.argmax(np.bincount(Y))
        return

    # 使用信息增益比选择最佳分裂属性
    purity_attrs = [purity_cal(X, Y, a) for a in attrs]
    purity_values = [pa[0] for pa in purity_attrs]
    split_values = [pa[1] for pa in purity_attrs]
    chosen_index = purity_values.index(max(purity_values))
    chosen_attr = attrs[chosen_index]

    root.attr = chosen_attr
    root.label = np.pi
    root.split_value = split_values[chosen_index]
    remaining_attrs = attrs.copy()
    del remaining_attrs[chosen_index]

    if root.split_value is None:  # 离散属性
        x_attr_col = X[:, chosen_attr]
        for x_v in set(X[:, chosen_attr]):
            n = Node(-1, -1, x_v)
            root.children.append(n)
            index_x_equal_v = np.where(x_attr_col == x_v)
            X_x_equal_v = X[index_x_equal_v]
            Y_x_equal_v = Y[index_x_equal_v]
            c45_tree_init(X_x_equal_v, Y_x_equal_v, remaining_attrs, n, purity_cal, 
                         depth + 1, max_depth, min_samples_split)
    else:  # 连续属性
        left_mask = X[:, chosen_attr] <= root.split_value
        right_mask = X[:, chosen_attr] > root.split_value
        left_child = Node(-1, -1, None)
        right_child = Node(-1, -1, None)
        root.children.append(left_child)
        root.children.append(right_child)
        c45_tree_init(X[left_mask], Y[left_mask], remaining_attrs, left_child, purity_cal, 
                     depth + 1, max_depth, min_samples_split)
        c45_tree_init(X[right_mask], Y[right_mask], remaining_attrs, right_child, purity_cal, 
                     depth + 1, max_depth, min_samples_split)

#### 1.2 基于C4.5决策树进行预测

##### **叶节点处理**

- 如果当前节点是叶节点（即 `tree_root.label != np.pi`），则直接返回当前节点的标签 `tree_root.label`，这是树的预测结果。
- 如果当前节点没有子节点，且 `tree_root.attr == np.pi`，则返回一个默认类别 `default_class`。

##### **连续属性处理**

- 如果当前节点的属性是连续的 (`tree_root.split_value is not None`)，则根据当前样本 `x` 对应属性的值与节点的分割点 `tree_root.split_value` 进行比较。
    - 如果属性值小于等于分割点，则递归地访问左子树（`tree_root.children[0]`）。
    - 否则，访问右子树（`tree_root.children[1]`）。
- 如果子节点不足，函数会返回默认类别。

##### **离散属性处理**

- 如果当前节点是离散属性节点（`tree_root.split_value is None`），则遍历所有子节点，检查当前样本 `x` 的属性值 `x[tree_root.attr]` 是否与子节点的属性值匹配。
    - 如果匹配，则递归地访问该子节点进行预测。

In [175]:
def c45_tree_predict(x, tree_root, default_class=0):
    """使用C4.5决策树进行预测"""
    if tree_root.label != np.pi:  # 如果是叶节点，直接返回类别
        return tree_root.label
    if tree_root.attr == np.pi or not tree_root.children:  # 如果无子节点，返回默认类别
        return default_class

    if tree_root.split_value is not None:  # 连续属性
        if len(tree_root.children) < 2:  # 子节点不足，防止越界
            return default_class
        if x[tree_root.attr] <= tree_root.split_value:
            return c45_tree_predict(x, tree_root.children[0], default_class)
        else:
            return c45_tree_predict(x, tree_root.children[1], default_class)
    else:  # 离散属性
        for child in tree_root.children:
            if child.attr_v == x[tree_root.attr]:
                return c45_tree_predict(x, child, default_class)
    return default_class

#### 1.3 可视化C4.5 决策树

In [176]:
def print_c45_tree(node, prefix="", is_last=True):
    """打印C4.5决策树结构"""
    if node.label != np.pi:  # 叶节点
        print(prefix + ("└── " if is_last else "├── ") + f"[叶节点]Label: {label_list[node.label]}")
        return

    # 内部节点
    feature_name = feature_list[node.attr]
    if node.split_value is not None:  # 连续值属性
        print(prefix + ("└── " if is_last else "├── ") + 
              f"[内部节点]Feature: {feature_name} (连续值, 分割点: {node.split_value:.3f})")
        for i, child in enumerate(node.children):
            is_last_child = (i == len(node.children) - 1)
            new_prefix = prefix + ("    " if is_last else "│   ")
            value_desc = "<= " if i == 0 else "> "
            value_desc += f"{node.split_value:.3f}"
            print(new_prefix + ("└── " if is_last_child else "├── ") + f"Value: {value_desc}")
            print_c45_tree(child, new_prefix + "    ", is_last_child)
    else:  # 离散值属性
        print(prefix + ("└── " if is_last else "├── ") + 
              f"[内部节点]Feature: {feature_name}")
        for i, child in enumerate(node.children):
            is_last_child = (i == len(node.children) - 1)
            new_prefix = prefix + ("    " if is_last else "│   ")
            value_desc = feature_dict[feature_name][child.attr_v] if feature_name in feature_dict else str(child.attr_v)
            print(new_prefix + ("└── " if is_last_child else "├── ") + f"Value: {value_desc}")
            print_c45_tree(child, new_prefix + "    ", is_last_child)

In [177]:
if __name__ == '__main__':
    # 加载数据
    ans = load_txt("Watermelon-train2.csv")
    X_train = ans[:, 1:-1]  # 特征矩阵
    Y_train = ans[:, -1].astype(np.int64)  # 标签

    test_data = load_txt("Watermelon-test2.csv")
    X_test = test_data[:, 1:-1]
    Y_test = test_data[:, -1].astype(np.int64)

    # 打印数据集基本信息
    print("\n数据集信息:")
    print(f"训练集样本数: {len(X_train)}")
    print(f"测试集样本数: {len(X_test)}")
    print(f"特征数量: {X_train.shape[1]}")
    print("\n特征列表:")
    for i, feature in enumerate(feature_list):
        print(f"特征 {i+1}: {feature}")

    # 初始化并训练决策树
    print("\n开始训练C4.5决策树...")
    r = Node(-1, -1, -1)
    attrs = list(range(X_train.shape[1])) 
    max_depth = 10
    min_samples_split = 2

    c45_tree_init(X_train, Y_train, attrs, r, gain_ratio, max_depth=max_depth, min_samples_split=min_samples_split)

    # 打印决策树结构
    print("\nC4.5决策树结构:")
    print_c45_tree(r)

    # 预测并评估
    y_predict = [c45_tree_predict(x, r) for x in X_test]
    accuracy = sum(1 for y_true, y_pred in zip(Y_test, y_predict) if y_true == y_pred) / len(Y_test)
    
    # 打印详细的预测结果
    print("\n测试集预测结果:")
    print("样本ID  实际类别  预测类别  是否正确")
    print("-" * 40)
    for i, (y_true, y_pred) in enumerate(zip(Y_test, y_predict)):
        is_correct = "✓" if y_true == y_pred else "✗"
        print(f"{i+1:^6d}  {label_list[y_true]:^8s}  {label_list[y_pred]:^8s}  {is_correct:^8s}")
    
    print(f"\n总体准确率: {accuracy:.2f}")


数据集信息:
训练集样本数: 17
测试集样本数: 5
特征数量: 4

特征列表:
特征 1: 色泽
特征 2: 根蒂
特征 3: 敲声
特征 4: 纹理

开始训练C4.5决策树...

C4.5决策树结构:
└── [内部节点]Feature: 纹理 (连续值, 分割点: 0.000)
    ├── Value: <= 0.000
        ├── [内部节点]Feature: 根蒂 (连续值, 分割点: 1.000)
        │   ├── Value: <= 1.000
        │       ├── [内部节点]Feature: 色泽 (连续值, 分割点: 0.000)
        │       │   ├── Value: <= 0.000
        │       │       ├── [叶节点]Label: 是
        │       │   └── Value: > 0.000
        │       │       └── [内部节点]Feature: 敲声 (连续值, 分割点: 0.000)
        │       │           ├── Value: <= 0.000
        │       │               ├── [叶节点]Label: 是
        │       │           └── Value: > 0.000
        │       │               └── [叶节点]Label: 是
        │   └── Value: > 1.000
        │       └── [叶节点]Label: 否
    └── Value: > 0.000
        └── [内部节点]Feature: 色泽 (连续值, 分割点: 1.000)
            ├── Value: <= 1.000
                ├── [内部节点]Feature: 敲声 (连续值, 分割点: 0.000)
                │   ├── Value: <= 0.000
                │       ├── [内部节点]Feature: 根蒂
  

#### **2. C4.5结果分析** 
总的来说，这个C4.5决策树模型表现不错，在测试集上取得了80%的准确率，具有良好的泛化能力。
##### 1. 决策树结构分析
- 根节点使用"纹理"作为首个分裂特征，这表明纹理是最具区分性的特征
- 第二层分别用了"根蒂"和"色泽"作为分裂特征
- 树的分支比较均衡，最大深度达到4层
- 多次使用了连续值分割点（如0.000，1.000等）

##### 2. 模型特点
- 模型能够同时处理离散特征和连续特征
- 对正反类别都有不错的预测能力

考虑增加训练样本数量以提高模型鲁棒性;可以尝试调整分裂阈值，看是否能提高准确率;下面进行剪枝处理，减小分支可能过于细致对结果的影响。




### **3. 高级要求**
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。



#### **3.1 代价复杂度剪枝（Cost-Complexity Pruning）**
代价复杂度剪枝（CCP）是一种常用的决策树剪枝方法，用于减少决策树的过拟合。它通过引入一个复杂度参数 `alpha`，在修剪决策树的同时考虑树的大小和误差，从而找到一个平衡点，既能保证较低的错误率，又能控制树的复杂度。CCP 通过计算每个节点剪枝后的效果，决定是否剪去某些节点，以减少模型的复杂度。

#### **剪枝算法步骤**

1. **节点遍历：** 遍历整个决策树的每个节点（通过 `collect_tree_nodes` 函数将节点收集到 `nodes` 列表中）。
   
2. **计算原始错误率：** 对整个树在验证集 `X_val` 和 `Y_val` 上进行预测，计算初始错误率 `best_error`。
   
3. **尝试剪枝：** 对每个非叶子节点，尝试进行剪枝，即将该节点的子节点删除，并将该节点的属性设置为 `np.pi`，表示该节点为叶节点。然后用该节点预测 `Y_val`，并计算剪枝后的错误率。
   
4. **计算代价：**
     $$
     \text{总代价} = \text{剪枝后的错误率} + \alpha \times \text{节点的样本数量}
     $$
     其中，$\alpha$ 是控制剪枝程度的复杂度参数。通过比较剪枝后的总代价与当前最小代价 `min_cost`，选择总代价较小的剪枝操作。
   
5. **剪枝决策：** 如果剪枝后的总代价比当前的误差更小，则执行剪枝，将该节点的子节点删除，并设置为叶节点。记录剪枝的节点数量。
   
6. **停止条件：** 当没有节点能够有效剪枝（即最佳剪枝的代价没有提高模型的表现时），算法停止。

#### **数学公式**

1. **错误率计算：**
   * 错误率通常用预测值与实际值的比较来定义，公式为：
     $$
     \text{error} = \frac{1}{n} \sum_{i=1}^{n} \mathbb{I}(y_i \neq \hat{y}_i)
     $$
     其中，$n$ 是样本数量，$y_i$ 是真实标签，$\hat{y}_i$ 是预测标签，$\mathbb{I}(x)$ 是指示函数，若 $x$ 为真则返回 1，否则为 0。

2. **剪枝后的总代价：**
     $$
     \text{total\_cost} = \text{error} + \alpha \times \text{node\_samples}
     $$
     其中，`node_samples` 是当前节点的样本数，$\alpha$ 是复杂度参数。

In [178]:
def cost_complexity_pruning(root, X_val, Y_val, alpha=0.01):
    """代价复杂度剪枝
    参数：
    - alpha: 复杂度参数，控制剪枝程度
    """
    nodes = []
    collect_tree_nodes(root, nodes)
    pruned_nodes = 0
    
    # 计算原始错误
    y_pred = predict_subtree(X_val, root)
    best_error = calculate_error_rate(Y_val, y_pred)
    
    while True:
        min_cost = float('inf')
        best_node = None
        best_label = None
        
        for node in nodes:
            if node.label != np.pi:  # 跳过叶节点
                continue
                
            # 保存原始状态
            original_label = node.label
            original_attr = node.attr
            original_children = node.children.copy()
            
            # 计算节点的样本数量
            y_pred_node = predict_subtree(X_val, node)
            node_samples = len(y_pred_node)
            
            # 尝试剪枝
            node.children = []
            node.attr = np.pi
            node.label = get_node_prediction(Y_val)
            
            # 计算剪枝后的错误率和代价
            y_pred_pruned = predict_subtree(X_val, root)
            error = calculate_error_rate(Y_val, y_pred_pruned)
            complexity_cost = alpha * node_samples
            total_cost = error + complexity_cost
            
            if total_cost < min_cost:
                min_cost = total_cost
                best_node = node
                best_label = node.label
            
            # 恢复节点
            node.label = original_label
            node.attr = original_attr
            node.children = original_children
        
        if best_node is None or min_cost >= best_error:
            break
            
        # 执行最佳剪枝
        best_node.children = []
        best_node.attr = np.pi
        best_node.label = best_label
        pruned_nodes += 1
        best_error = min_cost
        
    return pruned_nodes

#### **代价复杂度剪枝的优点**

* **控制过拟合：** 通过限制树的复杂度，可以有效防止过拟合，尤其是在样本较少时。
* **平衡错误率与树的复杂度：** 引入 $\alpha$ 参数后，可以根据需求调整模型的复杂度，使得模型既不过于简单，也不过于复杂。

In [179]:
# 主程序
print("构建原始决策树...")
root = Node(-1, -1, -1)
attrs = list(range(X_train.shape[1]))
c45_tree_init(X_train, Y_train, attrs, root, gain_ratio)

# 在剪枝前评估性能
print("\n剪枝前性能评估:")
y_pred_before = [c45_tree_predict(x, root) for x in X_test]
accuracy_before = sum(1 for y_true, y_pred in zip(Y_test, y_pred_before) if y_true == y_pred) / len(Y_test)
print(f"剪枝前测试集准确率: {accuracy_before:.3f}")

# 尝试不同的alpha值
alphas = [0.001, 0.005, 0.01, 0.05]
best_accuracy = accuracy_before
best_alpha = None
best_pruned_tree = None

for alpha in alphas:
    # 复制树进行剪枝尝试
    current_tree = copy.deepcopy(root)
    
    # 进行剪枝
    print(f"\n尝试 alpha = {alpha}")
    pruned_nodes = cost_complexity_pruning(current_tree, X_val, Y_val, alpha)
    
    # 评估剪枝后的性能
    y_pred = [c45_tree_predict(x, current_tree) for x in X_test]
    accuracy = sum(1 for y_true, y_pred in zip(Y_test, y_pred) if y_true == y_pred) / len(Y_test)
    print(f"剪枝掉 {pruned_nodes} 个节点")
    print(f"准确率: {accuracy:.3f}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_alpha = alpha
        best_pruned_tree = copy.deepcopy(current_tree)

# 输出最终结果
print("\n最终结果:")
final_tree = best_pruned_tree if best_pruned_tree is not None else root
if best_pruned_tree is not None:
    print(f"最佳alpha值: {best_alpha}")
    print(f"最佳准确率: {best_accuracy:.3f}")
    print(f"准确率提升: {(best_accuracy - accuracy_before):.3f}")
else:
    print("未找到更好的剪枝结果，保持原始树")

# 无论如何都输出详细预测结果
y_pred_final = [c45_tree_predict(x, final_tree) for x in X_test]
print("\n详细预测结果比较:")
print("样本ID  实际类别  剪枝前预测  剪枝后预测  剪枝前是否正确  剪枝后是否正确")
print("-" * 75)
for i, (y_true, y_pred_b, y_pred_f) in enumerate(zip(Y_test, y_pred_before, y_pred_final)):
    correct_before = "✓" if y_true == y_pred_b else "✗"
    correct_after = "✓" if y_true == y_pred_f else "✗"
    print(f"{i+1:^6d}  {label_list[y_true]:^8s}  {label_list[y_pred_b]:^8s}  "
          f"{label_list[y_pred_f]:^8s}  {correct_before:^12s}  {correct_after:^12s}")

构建原始决策树...

剪枝前性能评估:
剪枝前测试集准确率: 0.800

尝试 alpha = 0.001
剪枝掉 0 个节点
准确率: 0.800

尝试 alpha = 0.005
剪枝掉 0 个节点
准确率: 0.800

尝试 alpha = 0.01
剪枝掉 0 个节点
准确率: 0.800

尝试 alpha = 0.05
剪枝掉 0 个节点
准确率: 0.800

最终结果:
未找到更好的剪枝结果，保持原始树

详细预测结果比较:
样本ID  实际类别  剪枝前预测  剪枝后预测  剪枝前是否正确  剪枝后是否正确
---------------------------------------------------------------------------
  1        是         是         是           ✓             ✓      
  2        是         否         否           ✗             ✗      
  3        是         是         是           ✓             ✓      
  4        否         否         否           ✓             ✓      
  5        否         否         否           ✓             ✓      


#### **3.2 悲观剪枝算法（Pessimistic Pruning）**
悲观剪枝算法是一种通过考虑剪枝带来的潜在误差，并利用带惩罚项的错误率进行决策的剪枝方法。它通过对每个子树的剪枝后误差进行计算，估算剪枝的影响，并以此决定是否剪去某个节点。具体的策略是，如果剪枝后能够获得更低的错误率（包括惩罚项），则执行剪枝。

#### **算法步骤**

1. **递归遍历子树：** 从根节点开始，递归地遍历所有子树。

2. **计算不剪枝时的错误率：** 对每个子树，首先计算其错误率。如果不剪枝，计算该子树在验证集上的错误率 `subtree_error`。这里使用带惩罚项的错误率公式，来增加对错误率的悲观估计。
     $$
     \text{error\_rate} = \frac{n_{\text{errors}} + 0.5}{n_{\text{total}}}
     $$
     其中 $n_{\text{errors}}$ 是错误分类的样本数，$n_{\text{total}}$ 是总样本数。

3. **计算剪枝后的错误率：** 对于每个子树的节点，计算如果将该节点剪枝为一个叶节点，所产生的错误率 `leaf_error`。该叶节点的类别是多数类标签（`majority_class`）。
     $$
     \text{leaf\_error} = \frac{n_{\text{leaf\_errors}} + 0.5}{n_{\text{total}}}
     $$

4. **剪枝判断：** 如果剪枝后的错误率 `leaf_error` 小于或等于不剪枝时的错误率 `subtree_error`，则执行剪枝操作。具体操作是将当前节点的子节点删除，将其转变为叶节点，叶节点的类别为多数类标签。

5. **递归返回：** 如果剪枝成功，返回剪枝的节点数量；否则，继续递归处理子树。

#### **带惩罚项的错误率**

带惩罚项的错误率考虑了估算的不确定性。在样本量较小或错误率较高的情况下，直接使用错误率可能会导致过度拟合，因此加入了一个惩罚项来降低过度乐观的估计。

带惩罚项的错误率计算公式为：
$$
\text{standard\_error} = \sqrt{\frac{\text{error\_rate} \times (1 - \text{error\_rate})}{n_{\text{total}}}}
$$
最终的带惩罚错误率为：
$$
\text{error\_rate} + cf \times \text{standard\_error}
$$
其中，$cf$ 是惩罚系数，通常设为 0.25，用于控制标准误差对总错误率的影响。

In [180]:
def calculate_error_rate_with_penalty(n_errors, n_total, cf=0.25):
    if n_total == 0:
        return 1.0  # 错误率设为最大
    
    # 不使用Laplace校正
    error_rate = n_errors / n_total
    
    # 使用更准确的标准误差计算
    if error_rate == 0:  # 处理边界情况
        standard_error = np.sqrt(1.0 / (4 * n_total))
    else:
        standard_error = np.sqrt(error_rate * (1 - error_rate) / n_total)
    
    return error_rate + cf * standard_error

def pessimistic_pruning(root, X, Y, cf=0.25):
    if root is None or len(root.children) == 0:  # 修改叶子节点判断条件
        return 0
    
    pruned_nodes = 0
    
    # 先递归处理子树
    for child in root.children:
        pruned_nodes += pessimistic_pruning(child, X, Y, cf)
    
    # 获取当前子树的预测
    y_pred = predict_subtree(X, root)
    n_errors = np.sum(y_pred != Y)
    n_total = len(Y)
    
    # 计算不剪枝时的错误率
    subtree_error = calculate_error_rate_with_penalty(n_errors, n_total, cf)
    
    # 计算剪枝后的错误率
    majority_class = np.argmax(np.bincount(Y))
    leaf_errors = np.sum(Y != majority_class)
    leaf_error = calculate_error_rate_with_penalty(leaf_errors, n_total, cf)
    
    # 严格要求剪枝后错误率更低
    if leaf_error < subtree_error:
        root.children = []
        root.label = majority_class
        return pruned_nodes + 1
    
    return pruned_nodes

In [181]:
print("\n===== 开始对两种决策树进行剪枝实验 =====")

# 1. 首先加载两组数据集
# ID3使用的数据集
print("加载ID3数据集...")
id3_train_data = load_txt("Watermelon-train1.csv")
X_train_id3 = id3_train_data[:, 1:-1]
Y_train_id3 = id3_train_data[:, -1].astype(np.int64)
id3_test_data = load_txt("Watermelon-test1.csv")
X_test_id3 = id3_test_data[:, 1:-1]
Y_test_id3 = id3_test_data[:, -1].astype(np.int64)

# C4.5使用的数据集
print("加载C4.5数据集...")
c45_train_data = load_txt("Watermelon-train2.csv")
X_train_c45 = c45_train_data[:, 1:-1]
Y_train_c45 = c45_train_data[:, -1].astype(np.int64)
c45_test_data = load_txt("Watermelon-test2.csv")
X_test_c45 = c45_test_data[:, 1:-1]
Y_test_c45 = c45_test_data[:, -1].astype(np.int64)

# 在加载数据后添加
print(f"ID3训练集大小: {len(X_train_id3)}, 验证集大小: {val_size}, 测试集大小: {len(X_test_id3)}")
print(f"C4.5训练集大小: {len(X_train_c45)}, 验证集大小: {val_size}, 测试集大小: {len(X_test_c45)}")

# 2. ID3树剪枝实验
print("\n1. ID3树剪枝评估:")
id3_root = Node(-1, -1, -1)
id3_attrs = [0, 1, 2, 3]  # 离散属性
dicision_tree_init(X_train_id3, Y_train_id3, id3_attrs, id3_root, gain)

# 评估ID3原始性能
id3_predictions = [dicision_tree_predict(x, id3_root) for x in X_test_id3]
id3_accuracy = sum(1 for y_true, y_pred in zip(Y_test_id3, id3_predictions) if y_true == y_pred) / len(Y_test_id3)
print(f"ID3原始准确率: {id3_accuracy:.3f}")

# 对ID3树进行剪枝
best_id3_pruned = None
best_id3_accuracy = id3_accuracy
best_id3_cf = None

# 划分验证集
val_size = len(X_train_id3) // 5
X_val_id3 = X_train_id3[:val_size]
Y_val_id3 = Y_train_id3[:val_size]

for cf in [0.1, 0.25, 0.5, 0.75]:
    current_tree = copy.deepcopy(id3_root)
    pruned_nodes = pessimistic_pruning(current_tree, X_val_id3, Y_val_id3, cf)
    
    # 在原始测试集上评估
    y_pred = [dicision_tree_predict(x, current_tree) for x in X_test_id3]
    accuracy = sum(1 for y_true, y_pred in zip(Y_test_id3, y_pred) if y_true == y_pred) / len(Y_test_id3)
    
    print(f"惩罚系数 = {cf}: 剪掉 {pruned_nodes} 个节点, 准确率 = {accuracy:.3f}")
    
    if accuracy > best_id3_accuracy:
        best_id3_accuracy = accuracy
        best_id3_cf = cf
        best_id3_pruned = copy.deepcopy(current_tree)

# 3. C4.5树剪枝实验
print("\n2. C4.5树剪枝评估:")
c45_root = Node(-1, -1, -1)
c45_attrs = list(range(X_train_c45.shape[1]))
c45_tree_init(X_train_c45, Y_train_c45, c45_attrs, c45_root, gain_ratio)

# 评估C4.5原始性能
c45_predictions = [c45_tree_predict(x, c45_root) for x in X_test_c45]
c45_accuracy = sum(1 for y_true, y_pred in zip(Y_test_c45, c45_predictions) if y_true == y_pred) / len(Y_test_c45)
print(f"C4.5原始准确率: {c45_accuracy:.3f}")

# 对C4.5树进行剪枝
X_val_c45 = X_train_c45[:val_size]
Y_val_c45 = Y_train_c45[:val_size]

best_c45_pruned = None
best_c45_accuracy = c45_accuracy
best_c45_cf = None

for cf in [0.01,0.05,0.1, 0.25, 0.5, 0.75,1]:
    current_tree = copy.deepcopy(c45_root)
    pruned_nodes = pessimistic_pruning(current_tree, X_val_c45, Y_val_c45, cf)
    
    y_pred = [c45_tree_predict(x, current_tree) for x in X_test_c45]
    accuracy = sum(1 for y_true, y_pred in zip(Y_test_c45, y_pred) if y_true == y_pred) / len(Y_test_c45)
    
    print(f"惩罚系数 = {cf}: 剪掉 {pruned_nodes} 个节点, 准确率 = {accuracy:.3f}")
    
    if accuracy > best_c45_accuracy:
        best_c45_accuracy = accuracy
        best_c45_cf = cf
        best_c45_pruned = copy.deepcopy(current_tree)

# 4. 输出最终对比结果
print("\n===== 剪枝效果最终对比 =====")
print("\nID3决策树(基于Watermelon-train1/test1):")
print(f"原始准确率: {id3_accuracy:.3f}")
print(f"剪枝后准确率: {best_id3_accuracy:.3f}")
print(f"准确率变化: {best_id3_accuracy - id3_accuracy:.3f}")
print(f"最佳惩罚系数: {best_id3_cf}")

print("\nC4.5决策树(基于Watermelon-train2/test2):")
print(f"原始准确率: {c45_accuracy:.3f}")
print(f"剪枝后准确率: {best_c45_accuracy:.3f}")
print(f"准确率变化: {best_c45_accuracy - c45_accuracy:.3f}")
print(f"最佳惩罚系数: {best_c45_cf}")

print("\n分析总结:")
print("1. ID3树剪枝效果：", "提升" if best_id3_accuracy > id3_accuracy else "无提升或下降")
print("2. C4.5树剪枝效果：", "提升" if best_c45_accuracy > c45_accuracy else "无提升或下降")


===== 开始对两种决策树进行剪枝实验 =====
加载ID3数据集...
加载C4.5数据集...
ID3训练集大小: 16, 验证集大小: 3, 测试集大小: 10
C4.5训练集大小: 17, 验证集大小: 3, 测试集大小: 5

1. ID3树剪枝评估:
ID3原始准确率: 0.700
惩罚系数 = 0.1: 剪掉 2 个节点, 准确率 = 0.700
惩罚系数 = 0.25: 剪掉 2 个节点, 准确率 = 0.700
惩罚系数 = 0.5: 剪掉 2 个节点, 准确率 = 0.700
惩罚系数 = 0.75: 剪掉 2 个节点, 准确率 = 0.700

2. C4.5树剪枝评估:
C4.5原始准确率: 0.800
惩罚系数 = 0.01: 剪掉 2 个节点, 准确率 = 0.800
惩罚系数 = 0.05: 剪掉 2 个节点, 准确率 = 0.800
惩罚系数 = 0.1: 剪掉 2 个节点, 准确率 = 0.800
惩罚系数 = 0.25: 剪掉 2 个节点, 准确率 = 0.800
惩罚系数 = 0.5: 剪掉 2 个节点, 准确率 = 0.800
惩罚系数 = 0.75: 剪掉 2 个节点, 准确率 = 0.800
惩罚系数 = 1: 剪掉 2 个节点, 准确率 = 0.800

===== 剪枝效果最终对比 =====

ID3决策树(基于Watermelon-train1/test1):
原始准确率: 0.700
剪枝后准确率: 0.700
准确率变化: 0.000
最佳惩罚系数: None

C4.5决策树(基于Watermelon-train2/test2):
原始准确率: 0.800
剪枝后准确率: 0.800
准确率变化: 0.000
最佳惩罚系数: None

分析总结:
1. ID3树剪枝效果： 无提升或下降
2. C4.5树剪枝效果： 无提升或下降


#### **结果分析**

除了这两种剪枝算法，还尝试了REP (Reduced Error Pruning)、CVP (Critical Value Pruning)、MEP (Minimum Error Pruning)等算法。发现准确率并没有明显的提升，我们观察数据集也可以发现数据集规模太小，可能导致导致统计信息不可靠，剪枝决策可能不准确。

### **心得体会**
#### 实验过程中的挑战与解决

- **剪枝效果的评估**：在实验中，我发现剪枝的效果并不是简单的树变小或误差降低就能完全判断出来的。剪枝后的树是否能提高泛化能力，还是需要通过交叉验证等方式进行评估。因此，剪枝后的模型需要通过额外的验证集来测试其实际效果。
- **参数调优**：在调整惩罚系数时，我尝试了不同的参数值，并通过对比训练和验证误差的变化，找出了最优的剪枝参数。这个过程让我深刻理解了机器学习中“调参”的重要性。

#### 对决策树及其剪枝算法的深入理解

通过这次实验，我不仅掌握了C4.5决策树的构建过程，也深入了解了剪枝在提高模型精度和泛化能力方面的关键作用。决策树的剪枝方法可以有效地防止模型过拟合，特别是在数据量较大或者特征较多时，剪枝的效果尤为显著。

实际操作中，剪枝的主要目标是提高模型在新数据上的表现，而不仅仅是降低训练误差。这种转变让我对机器学习模型的评估和优化有了更加全面的理解。
